# SELECT AI

このノートブックでは、Autonomous Database で使用可能な [SELECT AI](https://docs.oracle.com/en/cloud/paas/autonomous-database/serverless/adbsb/sql-generation-ai-autonomous.html#GUID-9CE75F94-7455-4C09-A3F3-118C08E82B7E) を体験します。

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

# OCI
user_ocid = os.getenv("USER_OCID")
tenancy_ocid = os.getenv("TENANCY_OCID")
private_key_content = os.getenv("PRIVATE_KEY_CONTENT")
fingerprint = os.getenv("FINGERPRINT")
compartment_id = os.getenv("COMPARTMENT_ID")

# Oracle Database
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
dsn = os.getenv("DSN")
config_dir = os.getenv("CONFIG_DIR")
wallet_dir = os.getenv("WALLET_DIR")
wallet_password = os.getenv("WALLET_PASSWORD")
table_name = os.getenv("TABLE_NAME")

Oracle Database とのコネクションを作成します。  
Jupyter Notebook のため、`with` 句は利用していませんが、アプリケーションに組み込む際は、コネクションが確実にクローズされることを保証するために、`with` 句を用いてコネクションを作成ください。

参考: [https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html#closing-connections](https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html#closing-connections)

In [2]:
import oracledb

connection = oracledb.connect(
    dsn=dsn,
    user=username,
    password=password,
    config_dir=config_dir,
    wallet_location=wallet_dir,
    wallet_password=wallet_password
)

SELECT AI の対象となるサンプルテーブルとデータを作成します。

In [3]:
cursor = connection.cursor()

sample_data = [
    (1, "佐藤 太郎", "男性", "営業", "マネージャー", "経験豊富な営業マネージャー"),
    (2, "鈴木 次郎", "男性", "開発", "エンジニア", "AI開発に従事"),
    (3, "田中 花子", "女性", "人事", "リーダー", "採用業務を担当"),
    (4, "山田 太一", "男性", "マーケティング", "スペシャリスト", "デジタルマーケティング担当"),
    (5, "高橋 美咲", "女性", "開発", "エンジニア", "クラウドインフラのスペシャリスト"),
    (6, "井上 一郎", "男性", "営業", "チームリーダー", "営業部のチームリーダーで、大手企業向けの営業を担当"),
    (7, "小林 明美", "女性", "開発", "シニアエンジニア", "ソフトウェアアーキテクチャ設計の経験豊富"),
    (8, "森田 健太", "男性", "サポート", "カスタマーサポート", "製品サポートの専門家"),
    (9, "中村 優子", "女性", "経理", "経理担当", "会社全体の経理業務を担当"),
    (10, "藤田 一華", "女性", "営業", "アシスタント", "営業部のサポート業務を担当"),
    (11, "山本 翔太", "男性", "開発", "ジュニアエンジニア", "新卒エンジニアで、AIプロジェクトに参画中"),
    (12, "加藤 美和", "女性", "人事", "人事アシスタント", "人事データの管理や、採用サポートを担当"),
    (13, "佐々木 健", "男性", "マーケティング", "マーケティングアナリスト", "市場調査とデータ分析を担当"),
    (14, "斎藤 美紀", "女性", "営業", "フィールドセールス", "顧客訪問やプレゼンテーションを担当"),
    (15, "大野 智", "男性", "開発", "データサイエンティスト", "ビッグデータ分析と機械学習モデルの構築を担当"),
]

create_table_statement = """
CREATE TABLE IF NOT EXISTS EMPLOYEE (
    EMPLOYEE_ID    NUMBER PRIMARY KEY,       -- 社員番号
    NAME           VARCHAR2(100),            -- 従業員名
    GENDER         VARCHAR2(10),             -- 性別
    DEPARTMENT     VARCHAR2(50),             -- 部署
    POSITION       VARCHAR2(50),             -- 役職
    DETAILS        VARCHAR2(255)             -- 詳細情報
)
"""

insert_sample_data_statement = """
INSERT INTO EMPLOYEE (EMPLOYEE_ID, NAME, GENDER, DEPARTMENT, POSITION, DETAILS)
VALUES (:1, :2, :3, :4, :5, :6)
"""

# テーブル作成
cursor.execute(
    statement=create_table_statement
)

# データ初期化
cursor.execute(
    statement="""
    DELETE FROM EMPLOYEE
    """
)

# データ挿入
cursor.executemany(
    statement=insert_sample_data_statement,
    parameters=sample_data
)

cursor.close()
connection.commit()

（オプション日本語によるコメントを付与し、SQLの変換精度を向上させます。

In [4]:
cursor = connection.cursor()

comment_statement_table = """
COMMENT ON TABLE EMPLOYEE IS '従業員情報を管理しているテーブルです。'
"""
cursor.execute(statement=comment_statement_table)

comment_statement_employee_id = """
COMMENT ON COLUMN EMPLOYEE.EMPLOYEE_ID IS '従業員ID'
"""
cursor.execute(statement=comment_statement_employee_id)

comment_statement_name = """
COMMENT ON COLUMN EMPLOYEE.NAME IS '名前'
"""
cursor.execute(statement=comment_statement_name)

comment_statement_gender = """
COMMENT ON COLUMN EMPLOYEE.GENDER IS '性別'
"""
cursor.execute(statement=comment_statement_gender)

comment_statement_department = """
COMMENT ON COLUMN EMPLOYEE.DEPARTMENT IS '部署（開発、人事、営業、マーケティング、サポート、経理が存在します）'
"""
cursor.execute(statement=comment_statement_department)

comment_statement_position = """
COMMENT ON COLUMN EMPLOYEE.POSITION IS '役職'
"""
cursor.execute(statement=comment_statement_position)

comment_statement_details = """
COMMENT ON COLUMN EMPLOYEE.DETAILS IS '従業員の詳細情報'
"""
cursor.execute(statement=comment_statement_details)

SELECT AI で使用するための資格情報を作成します。

In [5]:
cursor = connection.cursor()

remove_credential_statement = """
BEGIN
       DBMS_CLOUD.DROP_CREDENTIAL (
              credential_name => :credential_name
       );
END;
"""

create_credential_statement = """
BEGIN
       DBMS_CLOUD.CREATE_CREDENTIAL (
              credential_name => :credential_name,
              user_ocid       => :user_ocid,
              tenancy_ocid    => :tenancy_ocid,
              private_key     => :private_key_content,
              fingerprint     => :fingerprint
       );
END;
"""

cursor.execute(
       statement=remove_credential_statement,
       credential_name="OCI_KEY_CRED",
)

cursor.execute(
       statement=create_credential_statement,
       credential_name="OCI_KEY_CRED",
       user_ocid=user_ocid,
       tenancy_ocid=tenancy_ocid,
       private_key_content=private_key_content,
       fingerprint=fingerprint
)

cursor.close()

SELECT AI 用の AI Profile を作成します。

In [6]:
import json

cursor = connection.cursor()

remove_ai_profile_statement = """
BEGIN
    DBMS_CLOUD_AI.DROP_PROFILE(
        profile_name => 'OCI_GENERATIVE_AI'
    );
END;
"""
cursor.execute(
    statement=remove_ai_profile_statement
)

create_ai_profile_statement = """
BEGIN
    DBMS_CLOUD_AI.CREATE_PROFILE(
        profile_name => 'OCI_GENERATIVE_AI',
        attributes   => :attributes
    );
END;
"""
attributes = {
    "provider": "oci",
    "credential_name": "OCI_KEY_CRED",
    "model": "cohere.command-r-plus",
    "oci_apiformat": "COHERE",
    "region": "us-chicago-1",
    "oci_compartment_id": compartment_id,
    "comments": True,
    "object_list": [
        {"owner": "SHUKAWAM", "name": "EMPLOYEE"},
    ]
}
cursor.execute(
    statement=create_ai_profile_statement,
    attributes=json.dumps(attributes)
)

set_ai_profile_statement = """
BEGIN
    DBMS_CLOUD_AI.SET_PROFILE(
        profile_name    => 'OCI_GENERATIVE_AI'
    );
END;
"""
cursor.execute(
    statement=set_ai_profile_statement
)

cursor.close()

SELECT AI 機能をいくつか試してみる

In [ ]:
cursor = connection.cursor()

select_ai_showsql = """
SELECT AI SHOWSQL 開発部の人は何人いますか？;
"""

cursor.execute(
    statement=select_ai_showsql
)

result = cursor.fetchall()
print(result)

cursor.close()

In [ ]:
cursor = connection.cursor()

select_ai_sql_narrate = """
SELECT AI NARRATE 従業員は何人いますか？;
"""

cursor.execute(
    statement=select_ai_sql_narrate
)

result = cursor.fetchall()
print(result)

cursor.close()

In [ ]:
cursor = connection.cursor()

select_ai_sql_chat = """
SELECT AI CHAT マネージャーの従業員は何人いますか？;
"""

cursor.execute(
    statement=select_ai_sql_chat
)

result = cursor.fetchall()
print(result)

cursor.close()

In [ ]:
cursor = connection.cursor()

select_ai_sql_run = """
SELECT AI RUN 従業員は何人いますか？;
"""

cursor.execute(
    statement=select_ai_sql_run
)

result = cursor.fetchall()
print(result)

cursor.close()

## w/ LangChain

LLMが使用するツールを定義します

In [26]:
from langchain_core.tools import tool, Tool

@tool
def nl_to_sql(query: str) -> list:
    """自然言語をSQLに変換（SELECT AI使用）し、そのSQLを実行し結果を返します"""
    with oracledb.connect(
        dsn=dsn,
        user=username,
        password=password,
        config_dir=config_dir,
        wallet_location=wallet_dir,
        wallet_password=wallet_password
    ) as connection:
        with connection.cursor() as cursor:
            set_ai_profile_statement = """
                BEGIN
                    DBMS_CLOUD_AI.SET_PROFILE(
                        profile_name    => 'OCI_GENERATIVE_AI'
                    );
                END;
            """
            cursor.execute(
                statement=set_ai_profile_statement
            )
            run_statement = f"select ai {query}"
            cursor.execute(statement=run_statement)
            result = cursor.fetchall()
            return result

In [27]:
from langgraph.prebuilt import ToolNode

tools = [
    Tool(
        name="nl_to_sql",
        func=nl_to_sql,
        description="""
            従業員テーブルに対する自然言語の問い合わせをSQLに変換したのち、そのSQLの実行結果を返します。
            特定の条件に当てはまる従業員を検索する際や、数の集計時に役にたつツールです。
        """
    )
]

tool_node = ToolNode(
    tools=tools,
    name="demo-tools",
    tags=["text-to-sql"]
)

LLMを定義します

In [28]:
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

compartment_id = os.getenv("COMPARTMENT_ID")
service_endpoint = os.getenv("SERVICE_ENDPOINT")

chat = ChatOCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
    model_id="cohere.command-r-plus",
    is_stream=True,
    model_kwargs={
        "temperature": 0,
        "max_tokens": 2500,
        "top_p": 0.75,
        "top_k": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
).bind_tools(tools=tools)

ちゃんと、該当のツールが使えていることを確認する

In [ ]:
response = chat.invoke("従業員って何人いますか？")

print(response.content)

Agentのノードを定義する

In [30]:
from typing import Literal, List

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState

def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state["messages"]
    response = chat.invoke(messages)
    return {"messages": [response]}

In [31]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")

In [32]:
workflow.add_conditional_edges("agent", should_continue)

workflow.add_edge("tools", "agent")

checkpointer = MemorySaver()

In [ ]:
app = workflow.compile(checkpointer=checkpointer)

app.get_graph().print_ascii()

In [ ]:
import uuid
from langchain_core.prompt_values import HumanMessage

session_id = str(uuid.uuid4())

result = app.invoke(
    input={
        "messages": [
            HumanMessage(content="従業員は何人いますか？")
        ]
    },
    config={
        "configurable": {
            "thread_id": session_id
        }
    }
)

print(result["messages"][-1].content)

## w/ Langfuse

In [35]:
from langfuse.callback import CallbackHandler

# Langfuse
secret_key = os.getenv("LANGFUSE_SECRET_KEY")
public_key = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_host = os.getenv("LANGFUSE_HOST")

langfuse_handler = CallbackHandler(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host,
    sample_rate=1.0
)

In [ ]:
session_id = str(uuid.uuid4())

result = app.invoke(
    input={
        "messages": [
            HumanMessage(content="従業員は何人いますか？")
        ]
    },
    config={
        "configurable": {
            "thread_id": session_id
        },
        "callbacks": [langfuse_handler]
    }
)

print(result["messages"][-1].content)

In [ ]:
session_id = str(uuid.uuid4())

result = app.invoke(
    input={
        "messages": [
            HumanMessage(content="開発部の人は何人いますか？")
        ]
    },
    config={
        "configurable": {
            "thread_id": session_id
        },
        "callbacks": [langfuse_handler]
    }
)

print(result["messages"][-1].content)